This notebook demonstrates the method to produce the most basic simulation under consideration using the package `varsim`. The inputs here are:
   - a set of pointings (for simplicity, read in from a csv file)
   - a (model, population) pair, where
       - a population of astrophysical sources described by a subclass of `varsim.BasePopulation`. This class must implement the abstract methods of `varsim.BasePopulation`, and provide an index for each source and the model parameters. 
       - a model for the sources described by a subclass of `varsim.BaseModel`. Again this subclass must implement all of the abstract methods and properties of `varsim.BaseModel`. The essential functionality of this class is to represent an astrophysical source as a model with model parameters, given which, this class has methods of predicting the model flux as a function of time at the top of the earth's atmosphere (ie. no sky noise included).
   - The simulation will be performed using the class `varsim.BasicSimulation` which is a subclass of the abstract base class `varsim.BaseSimulation` and implements concrete methods and properties necessary for the simulation. These methods and properties only use the abstract properties and methods of `BaseModel` and `BasePopulation`, and are therefore guaranteed to with any subclass.

##  Imports

In [1]:
import os

In [2]:
from opsimsummary import HealpixTiles, OpSimOutput

In [3]:
import numpy as np
import pandas as pd

In [4]:
from lsst.sims.photUtils import BandpassDict

In [5]:
from varsim import BasePopulation, BasicSimulation, BaseModel

In [6]:
from lsst.sims.catUtils.supernovae import SNObject

In [7]:
import analyzeSN as ans

In [8]:
import varsim

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

## The quantities of interest

####  Pointings

In [10]:
# The set of pointings randomly kept for convenience.
example_data = varsim.example_data
pointings_File = os.path.join(example_data, 'example_pointings.csv')
pointings = pd.read_csv(pointings_File, index_col='obsHistID')

In [11]:
# To show what this looks like
print(len(pointings))
pointings.head()

14


,sessionID,propID,fieldID,fieldRA,fieldDec,filter,expDate,expMJD,night,visitTime,...,moonBright,darkBright,rawSeeing,wind,humidity,slewDist,slewTime,fiveSigmaDepth,ditheredRA,ditheredDec
obsHistID,,,,,,,,,,,,,,,,,,,,,
1,1016,54,316,1.676483,-1.082473,y,2922,59580.033829,0,34.0,...,0.0,105.610378,0.920473,0.0,0.0,1.620307,0.000000,21.021236,1.643930,-1.108924
14,1016,54,1118,1.709686,-0.607725,y,3424,59580.039637,0,34.0,...,0.0,103.104582,0.903960,0.0,0.0,0.055856,4.599568,21.067249,1.691083,-0.634176
13,1016,54,1220,1.664386,-0.566519,y,3386,59580.039190,0,34.0,...,0.0,101.147431,0.903960,0.0,0.0,0.048200,4.776511,21.091149,1.646286,-0.592970
12,1016,54,1212,1.608166,-0.575819,y,3347,59580.038741,0,34.0,...,0.0,98.109397,0.903960,0.0,0.0,0.055665,4.739998,21.128319,1.589958,-0.602270
11,1016,54,1110,1.651280,-0.618574,y,3308,59580.038293,0,34.0,...,0.0,100.125895,0.903960,0.0,0.0,0.056173,4.499824,21.103635,1.632535,-0.645025


In [12]:
# We did not need to have all these columns. The essential columns are ra, dec, filter, 
# fivesigmadepth. It is also good to have fieldID, etc.

### Model of source
- Model : Subclass of `varsim.BaseModel` with its methods implemented. Here I use `lsst.sims.catUtils.supernovae.SNObject` functionality to provide  functions
- Provides `minMjd` and `maxMjd` to restrict the set of pointings over which we will build the light curve.
- `modelFlux`: concrete implementation of abstract method `BaseModel.modelFlux` providing the flux, given the model parameters and `mjd`, `bandpass`.

We will write down an example of a model as a concrete implementation of `BaseModel`. This is the minimal model required for describing a variable object for these simulations. Such an implementation requires the following properties to be implemented.

#### A SALT model

In [13]:
class Model(SNObject, BaseModel):
    def setModelParameters(self, params):
        paramDict = params.copy()
        # self.setCoords(ra=positions['ra'], dec=positions['dec'])
        self.set(**params)
        self.set_MWebv(0)
        #self.mwEBVfromMaps()
    
    @property
    def minMjd(self):
        return self.mintime()
    @property
    def maxMjd(self):
        return self.maxtime()
    
    def modelFlux(self, mjd, bandpassobj):
        return self.catsimBandFlux(bandpassobject=bandpassobj,
                                      time=mjd)



#### A Stupid Model
We will also work out the population for a stupid model to show every single step:
- The model is stupid becaue there are no k corrections or band dependence

In [15]:
from varsim import BaseModel

In [16]:
class StupidModel(BaseModel):
    def __init__(self):
        self.t0 = None
        self.f = None
        self._maxday = None
        self._minday = None
        self.z = NOne
        
    def setModelParameters(self, params):
        self.f = params['f']
        self.t0 = params['t0']
        self._maxday = params['max'] 
        self._minday = params['min']
        self.z = params['z']
        return
    @property
    def minMjd(self):
        return self._minMjd
    
    @property
    def maxMjd(self):
        return self._maxday + self.t0
    
    @property
    def minMjd(self):
        return self._minday + self.t0
    
    def modelFlux(self, mjd, bandpassobj):
        #d = mjd[(mjd > self.minMjd) & (mjd < self.maxMjd)]
        vals = np.sin((mjd - self.t0)/ self.f)
        return max(vals, 0)

### Population of astrophysical sources. 
Here I use supernovae with the SALT model.  I keep two supernovae in a completely random way without attempting to make sense. The important parts are  that no matter how the Population model is implemented, it has:
- `modelparams` : the method which takes the unique index of a supernova in the population and provides its model parameters as a dictionary. An important requirement is that the dictionary as the keys `ra`, `dec`, while the other parameters are completely user dependent
- `idxvalues` : a property which is a sequence of indices. Here the sequence is implemented as a tuple, which is perhaps how it should be for large simulations.

####  SALT model Population

In [17]:
class SALTPopulation(BasePopulation):
    def __init__(self):
        self.x0 = [5.0e-2, 3.e-5]
        self.x1 = [0, .1]
        self.c = [-0.2, 0.5]
        self.t0 = [59581., 59580. ]
        self.z = [0.5, 0.6]
        self.ra = [30., 30.]
        self.dec = [-45., -45.]
        
    def modelparams(self, idx):
        return dict(x0=self.x0[idx], x1=self.x1[idx], c=self.c[idx], t0=self.t0[idx],
                    z=self.z[idx])
    @property
    def idxvalues(self):
        return (x for x in (0, 1))
    @property
    def numSources(self):
        return sum(1 for i in self.idxvalues)
    #@property
    #def hasPositionArray(self):
    #    return False
    #@property
    #def positionArray(self):
    #    return None
    
    #def positions(self, idx):
    #    return dict(ra=self.ra[idx], dec=self.dec[idx])

####  Stupid model Population

In [18]:
class StupidPopulation(BasePopulation):
    def __init__(self):
        self.t0 = [59580.03, 59580.05]
        self.f = [0.002, 0.004]
        self.minvals = [-0.2, -0.5]
        self.maxvals = [0.5, 1.0]
        
    def modelparams(self, idx):
        return dict(t0=self.t0[idx], f=self.f[idx], min=self.minvals[idx], max=self.maxvals[idx])
    @property
    def idxvalues(self):
        return (x for x in (0, 1))
    @property
    def numSources(self):
        return sum(1 for i in self.idxvalues)
    #@property
    #def hasPositionArray(self):
    #    return False
    #@property
    #def positionArray(self):
    #    return None
    
    #def positions(self, idx):
    #    return dict(ra=self.ra[idx], dec=self.dec[idx])   

### Instantiate the populations and demonstrate the functionality

This is the Supernova model with no extinction

#### Population for SALT Model

In [19]:
sp = SALTPopulation()

In [ ]:
# The indices
idxs = tuple(sp.idxvalues)
print(idxs)

In [ ]:
# The model parameters
sp.modelparams(0)

and the other one

In [ ]:
sp.numSources

In [ ]:
sp.modelparams(0)

In [ ]:
model = Model()

In [ ]:
# Let us set parameters (using the method `setModelParameters` in the abstract class
# to the parameters of the first object in SALTParameters)
model.setModelParameters(sp.modelparams(0))    

In [ ]:
# Now, this should predict fluxes (in maggies) given a time, and a bandpassobject
f = (model.modelFlux(mjd=59580, bandpassobj=bandpassdict['y']), 
     model.modelFlux(mjd=59580, bandpassobj=bandpassdict['g']))
print(f, -2.5 * np.log10(f))

#### Population for Stupid Model

In [ ]:
stp = StupidPopulation()

In [ ]:
stp.t0

In [ ]:
stp.modelparams(0)

In [ ]:
# We will need the LSST bandpasses. Let us load them using the catsim method
bandpassdict = BandpassDict.loadTotalBandpassesFromFiles()

Similarly, we show the results for the `Stupid Model`

The stupid model:

In [ ]:
stupidParams = dict(t0=59580.03, f=0.001, min=-0.5, max=0.8)
sm = StupidModel()
sm.setModelParameters(stupidParams)

## Simulation
Now we are ready to do the simulation using the class `BasicSimulation`. This is a concrete implementation of `BaseSimulation`, 

In [ ]:
bsim = BasicSimulation(sp, model, pointings=pointings, rng=np.random.RandomState(0), 
                       maxObsHistID=1000000, pointingColumnDict=None,
                       pruneWithRadius=False)

In [ ]:
t_ = bsim.lc(1, maxObsHistID=2e6)

In [ ]:
t_

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(t_.expMJD, t_.flux, t_.fluxerr, fmt='o')

In [ ]:
sp.modelparams(0)

In [ ]:
!rm *.hdf

In [ ]:
bsim.write_simulation(phot_output='sim_phot_all.hdf', pop_output='sim_pop.hdf', method='hdf', key='sim1')

In [ ]:
phot_sim_df = pd.read_hdf('sim_phot_all.hdf', key='sim1')

In [ ]:
pop_sim_df = pd.read_hdf('sim_pop.hdf')

In [ ]:
phot_sim_df

In [ ]:
pop_sim_df

In [ ]:
stp.modelparams(0)

In [ ]:
sm.setModelParameters(stp.modelparams(0))

In [ ]:
sm.modelFlux(pointings.expMJD.iloc[0], bandpassobj=None)

In [ ]:
bstm = BasicSimulation(stp, sm, pointings=pointings, rng=np.random.RandomState(0), 
                       maxObsHistID=1000000, pointingColumnDict=None,
                       pruneWithRadius=False)
bstm.write_simulation(phot_output='stp_phot_all.hdf', pop_output='stm_pop.hdf', method='hdf', key='sim1')

In [ ]:
phot_simt_df = pd.read_hdf('stp_phot_all.hdf', key='sim1')

In [ ]:
pop_simt_df = pd.read_hdf('stm_pop.hdf')

In [ ]:
pop_simt_df

In [ ]:
bsim.lc(0)

In [ ]:
bstm.lc(0)

In [ ]:
phot_simt_df

# Scratch

In [ ]:
from varsim import BaseSpatialPopulationFromStochasticModel, BaseSpatialPopulation

In [ ]:
class SpatialPopulation()

In [ ]:
b = BaseModel()

In [ ]:
from varsim import BasePopulationFromStochasticModel

In [ ]:
class SALTStatPopulation(BasePopulationFromStochasticModel):
    def __init__(self, ):
        self.x0 = [5.0e-2, 3.e-5]
        self.x1 = [0, .1]
        self.c = [-0.2, 0.5]
        self.t0 = [59581., 59580. ]
        self.z = [0.5, 0.6]
        self.ra = [30., 30.]
        self.dec = [-45., -45.]
    
    @property
    def pop_rng(self):
        return 0
    
    @property
    def numSources(self):
        return sum(1 for elem in self.idxvalues)
    #def get_pop_rng(self):
    #    return self.pop_rng
    def set_pop_rng(self, value):
        self._pop_rng = value
        
    def modelparams(self, idx):
            return dict(x0=self.x0[idx], x1=self.x1[idx], c=self.c[idx], t0=self.t0[idx],
                        z=self.z[idx])
        
    @property
    def idxvalues(self):
        return (x for x in (0, 1))
    @property
    def hasPositionArray(self):
        return False
    @property
    def positionArray(self):
        return None
    def positions(self, idx):
        return dict(ra=self.ra[idx], dec=self.dec[idx])
    

In [ ]:
snsims = SALTPopulation()

In [ ]:
bsim.lc(1).index.values.size

In [ ]:
np.unique(bsim.lc(0).index.values).size

In [ ]:
bsim.lc(0).columns

In [ ]:
bsim.write_lc(0, 'test_0.hdf', 'hdf')

In [ ]:
lc_0 = pd.read_hdf('test_0.hdf')

In [ ]:
lc_0

In [ ]:
from snsims import PowerLawRates

In [ ]:
pr = PowerLawRates(np.random.RandomState(0), numBins=28, fieldArea=1.0)

In [ ]:
np.shape(pr.zSamples)

In [ ]:
30 * 25000 / 1000

In [ ]:
class SaltPopulation(BasePopulationFromStochasticModel):
    def __init__(self, saltdist, tiling):
        

In [ ]:
from astropy.cosmology import Planck15

In [ ]:
from opsimsummary import Tiling

In [ ]:
dir(Tiling)

In [ ]:
from opsimsummary import HealpixTiles, Tiling

In [ ]:
hpTiles = HealpixTiles(nside=256,
                       preComputedMap='/Users/rbiswas/data/LSST/OpSimData/healpixelized_MINION_1016_256.db')

In [ ]:
r = hpTiles.positions(0, 1000)

In [ ]:
r[0]

In [ ]:
class TiledPopulation(BasePopulationFromStochasticModel, Tiling):
    def __init__(self):
        return super(self.__class__).__init()

In [ ]:
tp = TiledPopulation()

In [ ]:
class HPTP(SALTPopulation):
    def __init__(self):
        return super(self.__class__, self).__init__()

In [ ]:
hptp = HPTP()

In [ ]:
hptp.area(0)

In [ ]:
from snsims import SALT2_MMDist

In [ ]:
mB, x1, c, z = SALT2_MMDist(10000)

In [ ]:
fig, ax = plt.subplots()
ax.hist(c)

In [ ]:
from astropy.cosmology import Planck15

In [ ]:
pr.DeltaT

In [ ]:
class SALT2Params(BasePopulationFromStochasticModel):
    def __init__(self, zSamples, mjdmin=0., snids=None,
                 Mdisp=0.15, alpha=0.11, beta=-3.14, positions=None,
                 rng=None, cosmo=Planck15):
        self.zSamples = zSamples
        self._snids = snids
        self.alpha = alpha
        self.beta = beta
        self._numSN = len(self.zSamples)
        self._rng = rng
        self.Mdisp = Mdisp
        self.cosmo = cosmo
        self.mjdmin = mjdmin
        self.surveyDuration = self.zSamples.DeltaT * 365.0 
        self._positions = positions
    
    @property
    def mjdmax(self):
        return self.mjdmin + self.surveyDuration
    
    @property
    def numSources(self):
        return self._numSN
   
    @property
    def pop_rng(self):
        return self._rng
    
    def set_pop_rng(self, value):
        self._rng = value
    
    @property
    def idxvalues(self):
        if self._snids is None:
            self._snids = np.arange(self.numSN)
        return self._snids
    
    @property
    def positionArray(self):
        return None
    @property
    def hasPositionArray(self):
        return False
    
    def positions(self, idx):
        return dict(ra=self._positions[0][idx], dec=self._positions[1][idx])
    
    def snparamTable(self):
        if self._paramSamples is not None:
            return self._paramSamples
        timescale = self.mjdmax - self.mjdmin
        T0Vals = self.randomState.uniform(size=self.numSN) * timescale \
            + self.mjdmin
        mB, x1, c, m = SALT2_MMDist(self.numSN)
        x0 = np.zeros(len(mB))
        mB += self.randomState.normal(loc=0., scale=self.Mdisp,
                                      size=self.numSN)
        model = sncosmo.Model(source='SALT2')
        for i, z in enumerate(self.zSamples):
            model.set(z=z, x1=x1[i], c=c[i])
            model.source.set_peakmag(mB[i], 'bessellB', 'ab')
            x0[i] = model.get('x0')
        df = pd.DataFrame(dict(x0=x0, x1=x1, c=c, mB=mB, z=self.zSamples))
        return df
    def modelparams(self, idx):
        return self.snparamTable.ix(idx)
        
        
        
    
        
        
        
    

In [ ]:
sp = SALT2Params(pr.zSamples, 59580)

In [ ]:
bsim.write_simulation('test_sim.hdf', 'hdf', key='test', clobber=False)

In [ ]:
simdf = pd.read_hdf('test_sim.hdf')

In [ ]:
simdf

In [ ]:
bsim.write_population()

In [ ]:
df = pd.read_hdf('test.hdf', key='test')

In [ ]:
bsim.lc(1).to_hdf('test.hdf', key='test', mode='w', append=True, format='t')

In [ ]:
bsim.lc(0).to_hdf('test.hdf', key='test', mode='a', append=True, format='t')

In [ ]:
df = pd.read_hdf('test.hdf', key='test')

In [ ]:
df

In [ ]:
bsim.write_lc(0, 'test_0.csv', 'csv', key=None)

In [ ]:
dfcsv = pd.read_csv('test_0.csv')

In [ ]:
bsim.write_simulation('test.hdf', 'hdf', key=None)

In [ ]:
df = pd.read_hdf('test.hdf', key='0')

In [ ]:
df.to_sql('')

In [ ]:
bsim.write_population('pop.hdf', method='hdf')

In [ ]:
sp.idxvalues

In [ ]:
dfcsv.equals(df, )

In [ ]:
bsim = BasicSimulation(population=sp, model=Model, pointings=)

In [ ]:
bsim.write_lc(0, 'lc_0.hdf', 'hdf')

In [ ]:
pd.DataFrame.to_hdf()

In [ ]:
bsim.write_simulation('test.hdf', 'hdf', key='test')

In [ ]:
simdf = pd.read_hdf('test.hdf', key='test')

In [ ]:
simdf

In [ ]:
sp.positions

In [ ]:
model = Model()

In [ ]:
sp.modelparams(0)

In [ ]:
sp.modelparams(0)

In [ ]:
model.setModelParameters(**sp.modelparams(0))

In [ ]:
model.sn.SNstate

In [ ]:
sn = SNObject()

In [ ]:
params

In [ ]:
sn.setCoords(params['ra'], params['dec'])

In [ ]:
sn.set(**params)

In [ ]:
sn.mwEBVfromMaps()

In [ ]:
sn.SNstate

In [ ]:
model = Model()

In [ ]:
params = sp.modelparams(0).copy()

In [ ]:
for key in ('ra', 'dec'):
    params.pop(key)

In [ ]:
params

In [ ]:
sn.set(**params)

In [ ]:
sn.SNstate

In [ ]:
model.setModelParameters(**sp.modelparams(0))

In [ ]:
model.sn.SNstate

In [ ]:
bandpassobj = bandpassdict['y']

In [ ]:
model.setModelParameters(**sp.modelparams(0))

In [ ]:
model.modelFlux(59580., bandpassobj=bandpassobj)

In [ ]:
bsim = BasicSimulation(sp, model, pointings, rng=np.random.RandomState(0), 
                       maxObsHistID=1000000, pointingColumnDict=None,
                       timeRange=
                      pruneWithRadius=False)

In [ ]:
bsim.model.maxMjd

In [ ]:
bsim.model.r

In [ ]:
scrtch__

In [ ]:
opsout = _OpSimOutput.fromOpSimDB('/Users/rbiswas/data/LSST/OpSimData/minion_1016_sqlite.db',
                                 )

In [ ]:
pointings = opsout.summary.query('expMJD < 59580.04')

In [ ]:
pointings.expMJD.min()

In [ ]:
len(pointings)

In [ ]:
pointings.to_csv('check_poinx``tings.csv')

In [ ]:
!head check_pointings.csv

In [ ]:
from var

In [ ]:
from varsim import BaseExcep

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
df = pd.DataFrame()

In [ ]:
df['objid'] = np.arange(10)

In [ ]:
df

In [ ]:
280 *2 /4 + 20

In [ ]:
from collections import namedtuple, OrderedDict as odict

In [ ]:
keys = 'abcde'
vals = np.arange(5)
idx = 'pq'
s1 = (dict( (k, v) for (i, k,v) in zip(idx, keys, vals)))
s2 = (dict( (k, v) for (i, k,v) in zip(idx, keys, vals)))



In [ ]:
s = namedtuple(idx, s1)

In [ ]:
s = list()
_ = list(s.append(x) for x in (s1, s2))

In [ ]:
s1

In [ ]:
xx = list()

In [ ]:
xx = tuple(s)

In [ ]:
xx.append(s)

In [ ]:
idx = (l for l in idx)

In [ ]:
df = pd.DataFrame(s, index=idx)

In [ ]:
df

In [ ]:
df = pd.DataFrame()

In [ ]:
type(s)

In [ ]:
df.append(s)

In [ ]:
365*3 *2

In [ ]:
from sqlalchemy import create_engine

In [ ]:
conn = create_engine('sqlite:////Users/rbiswas/data/LSST/OpSimData/minion_1016_sqlite.db')

In [ ]:
from opsimsummary import HealPixelizedOpSim

In [ ]:
HealPixelizedOpSim()

In [ ]:
pd.read_sql_table()

In [ ]:
pd.read_sql_query('SELECT * FROM Summary WHERE PROPID == 56', conn)